## What does our ZKP look like 

We have a bunch of stuff 

1. A witness which is information that lets us make the proof in our case this is knowledge of a variable x that satisfies our eqation. It also includes the intermediate values that are zkp uses. This is secret to the user they want to prove this information.
2. A set of gate constraints. Basically that all the multiplicaions and addition we do are correct. 
3. A copy constriant check
4. Input / output checks

# Plonk Tutorial
This tutorial is based upon https://www.vitalik.ca/general/2019/09/22/plonk.html it expands upon the ideas described there and teaches the user to build their own plonk implemenation in python.

This tutorial takes to forum of a series of challenges where the user eventually build a plonk implmentation of a single proof. 

Plonk allows us to make arbitary zero knoledge proofs. For the purposes of this tutorial we will prove that we know an x such that  $P(x) = x^3 + x + 5 = 35$ this is a toy problem

![](https://vitalik.ca/files/posts_files/plonk-files/Circuit.png)

You can see we have two kinds of constraints gate constraints and copy constraints. A constraint is like an assertion from python. The program can only continue running if this assertion is true.

We will first handle the gate constraints and then tackle the copy constraints. 

## Gen witness

So lets first find a satisfying solution to the probelm we are trying to make proofs about $x^3 + x + 5 = 35$

The variables a, b and c will be the checking of additions / multiplicaions operatirons. Where we define `a + b == c` or `a * b == c`

In [1]:


def gen_witness(x, y, z):
    a = [x,     y,      z,      x*x]
    b = [x,     y,      z,      y*y]
    c = [x*x,   y*y,    z*z,    z*z]
    return(a,b,c)

Right so now that we have a, b , c we are ready to test. `is_satisfied` tests that our witness matches the constraints we are planing to add. 

Basically a[0] * b[0] = c[0] check this is a multiplicaion
a[2] + b[2] = c[2] check this is an addition.

In [2]:
def is_satisfied_witness(a, b, c):
    assert(a[0] * b[0] == c[0])
    assert(a[1] * b[1] == c[1])
    assert(a[2] * b[2] == c[2])
    assert(a[3] + b[3] == c[3])

In [3]:
# from plonk.sample_problem import gen_witness, is_satisfied_witness

a, b , c = gen_witness(3, 4, 5)

# Uncomment the next line and run
is_satisfied_witness(a,b,c)

Reader should investigate why this fails and why the next passes

In [4]:
a, b , c = gen_witness(1, 2, 3)
is_satisfied_witness(a,b,c)

AssertionError: 

## In plonk everything is a polynomial

In the previous section we generated our witness. A witness is a valid solution to our constraints. Where here our constraints are $x^3 + x + 5 == 35$

Next we want to define the actual constraints. They will be defined as a polynomial. Lets start out by creating a function eval_poly which takes a polynomial and evaluates it at a given point. Take the polynomial $1 + x + x^2 = y$ which is defined by this list [1,1,1] from lowest degree (ie starting at the $1*x^0$ ) to highest (1*x^2) 

In [6]:
## User input here.
def eval_poly(coef, x):
    res = []
    power = 1
    for i in coef:
        res.append(i * power)
        power = power * x
    return(round(sum(res)))

In [7]:
assert(eval_poly([1,1,1], 2) == 7 )
assert(eval_poly([-2, 7, -5, 1], 0) == -2)
assert(eval_poly([-2, 7, -5, 1], 1) == 1)
assert(eval_poly([-2, 7, -5, 1], 2) == 0)
assert(eval_poly([-2, 7, -5, 1], 3) == 1)

Okay now it seems out polynomial evaluations are working :)

Our mul / add constraint is defined by $\left(Q_{L_{i}}\right) a_{i}+\left(Q_{R_{i}}\right) b_{i}+\left(Q_{O_{i}}\right) c_{i}+\left(Q_{M_{i}}\right) a_{i} b_{i}+Q_{C_{i}}=0$ we can use this to check additions and multiplications. Define the constraint polynomial.


In [8]:
def constraint_polynomial(Qli, Qri, Qmi, Qoi, Qci, ai, bi, ci):
    return(Qli*ai + Qri*bi + Qoi*ci + Qmi*ai*bi + Qci == 0)

def validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c):
    for Qli,Qri,Qmi,Qoi,Qci,ai,bi,ci in zip (Ql,Qr,Qm,Qo,Qc,a,b,c):
        if (constraint_polynomial(Qli,Qri,Qmi,Qoi,Qci,ai,bi,ci) == False):
            return(False)
    return(True)

In [9]:
def test_addition():
    # constraints
    Ql = [1]
    Qr = [1]
    Qm = [0]
    Qo = [-1]
    Qc = [0]

    # witness
    a = [0]
    b = [1]
    c = [1]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)


def test_mul():

    # constraints
    Ql = [0]
    Qr = [0]
    Qm = [1]
    Qo = [-1]
    Qc = [0]

    # witness
    a = [1]
    b = [1]
    c = [1]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)



def test_constant():
    # constraints
    Ql = [1]
    Qr = [1]
    Qm = [0]
    Qo = [0]
    Qc = [-10]

    # witness
    a = [10]
    b = [0]
    c = [10]

    assert ( validate_native(Ql, Qr, Qm, Qo, Qc, a, b, c) == True)

test_addition()
test_mul()
test_constant()



Okay so now we are doing multiplicaions and additions. We can validate manually that all of these are being done correctly. Right so this is working we can make constraints. So lets make all the constraints for our system. First lets make some helpers that drop the constraints where we need then make sure they pass the tests :)

In [10]:
from plonk.constraint import add_add_constarint, add_mul_constarint, add_constant_constraint

Okay now lets add the actual constraints. By setting Ql, Qr, Qm , Qo and Qc such that it evaluates to a multipicion constaint at Ql[0] and an addition at Ql[2]. 

In [11]:
def gen_constraints():
    # Prove that I know an X such that x*x + y*y = z*z

    """
    Ql  a   +  Qr   b   + Qo   c   + Qm   a*b  + Qc    = 0
    1 * x   *   1 * x   + -1 * x*x   ~    ~      ~     = 0
    1 * y   *   1 * y   + -1 * y*y   ~    ~      ~     = 0
    1 * z   *   1 * z   + -1 * z*z   ~    ~      ~     = 0
    1 * x*x +   1 * y*y +  ~   ~     ~    ~      -z*z  = 0
    """

    # init constraints
    Ql = [1, 1, 1, 1]
    Qr = [1, 1, 1, 1]
    Qm = [0, 0, 0, -1]
    Qo = [-1, -1, -1, 0]
    Qc = [0, 0, 0, 0]

    # set constraints
    # Ql, Qr, Qm, Qo, Qc = add_mul_constarint(Ql, Qr, Qm, Qo, Qc)
    # Ql, Qr, Qm, Qo, Qc = add_mul_constarint(Ql, Qr, Qm, Qo, Qc)
    # Ql, Qr, Qm, Qo, Qc = add_add_constarint(Ql, Qr, Qm, Qo, Qc)
    # Ql, Qr, Qm, Qo, Qc = add_constant_constraint(Ql, Qr, Qm, Qo, Qc, 5)
    # # todo add a constant constraint for 1
    # Ql, Qr, Qm, Qo, Qc = add_add_constarint(Ql, Qr, Qm, Qo, Qc)
    return(Ql, Qr, Qm, Qo, Qc)

## Copy constraints 
So at the moment the system is not secure. Basically we are checking that the variables at location 

1. `a[0] * b[0] == c[0]` 
2. `a[1] * b[1] == c[1]`
3. `a[2] + b[2] == c[2]`


But we are just hoping that `a[1] == c[0]` we need to add constraints to make sure that we copy c[0] to a[1] these are called copy constraints you may also have heard of them refered to as permutation arguments. 

TODO: Do the actual attack

The naive thing to do is to do these checks manually. Basically make sure that each variable is equal to the other. The probelm with this is that it means that we need to check every variable which breaks privacy and succintness. Instead we will find a way to do this check using polynomials. 

Right now we have our witness which is `witness = a + b + c` and we want to prove that the value at `witness[0] == witness[8] == witness[7] == witness[6]` all of these values corresponding to our initial x. 

So now we need to make 3 polynomials the first to return the index of witness we want to look up, `witness_x_1`. The second `witness_x_2` to return the index after the permutation has been applied. And the third witness_y which returns the actual value of that witness at a given index. 

Write code that returns all of these. 

hint: use `from scipy.interpolate import lagrange` to interpolate a polynomial that passes through several points. 

In [12]:
%pip install scipy

You should consider upgrading via the '/Users/marcuspang/Desktop/Coding/pba/group-assignment-4/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Lets do the first. write fird permutation yourself
from plonk.copy_constraint import find_permutation
from plonk.poly import polynomial_eval

witness = a + b + c
eval_domain = range(0,len(witness))
witness_x_a = find_permutation(range(0,len(a)), range(0,len(a)))
witness_x_b = find_permutation(range(len(a),len(b)*2), range(len(a),len(b)*2))
witness_x_c = find_permutation(range(len(b)*2, len(c)*3), range(len(a)*2,len(a)*3))

witness_y = find_permutation(witness, eval_domain)

for i, val in enumerate(witness):
    assert(val == polynomial_eval(witness_y, i))


The test code below checks that witness_y returns the same results when we use the permutated indexes or the non permuated version. This means that each value there matches. 

In [14]:
# Okay now lets rearrange it so that the values get swapped when they match
# from plonk.sample_problem import gen_copy_constraints

def gen_copy_constraints():

    # copy constraints
    # a = [x , x*x, x*x*x, 1,  1, x*x*x + x]
    # b = [x , x, x, 5, 35, 5]
    # c = [x*x, x*x*x , x + x*x*x ,5, 35, 35]
    # inputs  = [x , x*x, x*x*x, 1,  1,
    #           x*x*x + x, x , x, x, 5, 35, 5
    #           x*x, x*x*x , x + x*x*x ,5, 35, 35]

    """
    Ql  a   +  Qr   b   + Qo   c   + Qm   ab  +  Qc = 0
    1 * x   *   1 * x   + -1 * x*x   ~    ~      ~  = 0
    1 * y   *   1 * y   + -1 * y*y   ~    ~      ~  = 0
    1 * z   *   1 * z   + -1 * z*z   ~    ~      ~  = 0
    1 * x*x +   1 * y*y + -1 * z*z   ~    ~      ~  = 0

    init constraints
    Ql = [1, 1, 1, 1]
    Qr = [1, 1, 1, 1]
    Qm = [0, 0, 0, -1]
    Qo = [-1, -1, -1, 0]
    Qc = [0, 0, 0, 0]

    a = [x, y, z, x*x]
    b = [x, y, z, y*y]
    c = [x*x, y*y, z*z, 0]

    copy_constraints = [x, y, z, x*x, x, y, z, y*y, x*x, y*y, z*z, 0]
    """
    copy_constraints = [4, 5, 6, 8, 0, 1, 2, 9, 3, 7, 11, 10]
    # copy_constraints = [8, 12, 13, 3, 4, 14, 0, 6, 7, 15, 17, 9, 1, 2, 5, 11, 10, 16]

    eval_domain = range(0, len(copy_constraints))

    x_a_prime = find_permutation(copy_constraints[0:4], eval_domain[0:4])
    x_b_prime = find_permutation(copy_constraints[4:8], eval_domain[4:8])
    x_c_prime = find_permutation(copy_constraints[8:12], eval_domain[8:12])

    return (x_a_prime, x_b_prime, x_c_prime, copy_constraints)

witness_x_a_perm, witness_x_b_perm, witness_x_c_perm, copy_constraints = gen_copy_constraints()
copy_constraints
for i in range(0,len(a)):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_a, i)) ==
           polynomial_eval (witness_y ,polynomial_eval(witness_x_a_perm, i)))

for i in range(len(a), len(a)*2):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_b, i)) ==
           polynomial_eval (witness_y ,polynomial_eval(witness_x_b_perm, i)))

for i in range(len(a)*2, len(a)*3):
    assert(polynomial_eval(witness_y , polynomial_eval(witness_x_c, i)) ==
           polynomial_eval (witness_y ,polynomial_eval(witness_x_c_perm, i)))


So now we have a way of checking permutations with polynomials. But we still need to check every variable which means we have not really gained anything. So next we will embed these three polynomials in a third such that we can check batches of permutations at once. 

To do this we take a random linear combination of witness_x_1 and witness_y. $rlc = v1 + wintess_{x_1} + v2*witness_y$

Then we calculate P(x) where P(0) = 1 and $P(x+1) = p(x)* rlc$

Then we do the same for witness_x_2 calculating P_2(x). Because v1 and v2 are random numbers we know that P_1(i) == P_2(i) if and only if witness_Y gives the same results when evaluated on witness_x_1(0:i) and witness_x_2(0:i)

In [15]:

from plonk.copy_constraint import copy_constraint_simple

# we have to generate v1 and v2 after a, b and c have been fixed.
v1 = hash(str(a + b + c))
v2 = hash(str(c + a + b))

eval_domain = range(0, len(a)*3)


x, Y , Px_a, rlc = copy_constraint_simple(range(0,len(a)), witness_x_a, witness_y, v1, v2)
x, Y , Px_b, rlc = copy_constraint_simple(range(len(a),len(a)*2), witness_x_b, witness_y, v1, v2)
x, Y , Px_c, rlc = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_c, witness_y, v1, v2)

# calcualte permutated polynomial
x_1, Y_1 , Px_a_prime, rlc_1 = copy_constraint_simple(range(0,len(a)), witness_x_a_perm, witness_y, v1, v2)
x_1, Y_1 , Px_b_prime, rlc_1 = copy_constraint_simple(range(len(a),len(a)*2), witness_x_b_perm, witness_y, v1, v2)
x_1, Y_1 , Px_c_prime, rlc_1 = copy_constraint_simple(range(len(a)*2,len(a)*3), witness_x_c_perm, witness_y, v1, v2)

assert(Px_a[-1] * Px_b[-1] * Px_c[-1] == Px_a_prime[-1] * Px_b_prime[-1] * Px_c_prime[-1])
assert(Px_a[0] == Px_b[0] == Px_c[0] == Px_a_prime[0] == Px_b_prime[0] == Px_c_prime[0] == 1 )

So now we can evaluate many copy constraints by simply checking a single point. But the problem is that the verifier needs to compute the Px_a ... Px_c_prime. We want to come up with a way so that they don't need to evaluate these instead letting the prover produce an argument that they have evaluated them correctly and minimize the verifiers work. We will do that in after the next section. In the next section we will make a quick fft sidetrack cos we need that to make a performant prover. 


## Part x: FFT

So you can see that it takes about 1.5 seconds for 100 points. In reality we will want to make proofs for systems that constain orders of magnatudes more variables in less than that time. So lets use FFT to speed it up. 

So this is based upon https://vitalik.ca/general/2019/05/12/fft.html which is good to read before you continue. 

TODO: possibly break this into a seperate tutorial

Firstly fft stands for fast foruier transform. A foruier transform is basically evaluating a polynomial. There are two ways to represent polynomials 

1. Is via coefficients [1,0,3] is $1 + x^2$ where its represented by coefficients 
2. Is with evaulations calculate the foruier space (evaulation space) values. 



In [16]:
from plonk.poly import polynomial_eval

poly = [1,0,3]
fs = []
fs.append(polynomial_eval(poly,0))
fs.append(polynomial_eval(poly,1))
fs.append(polynomial_eval(poly,2))
print(fs)


[1, 4, 13]


Both coefficient space and evaluation space uniquely identify the polynomial as long as it has been evaluated at a few positions. So what we did by evaluating the polynomial is a fourier transform. It is also possible to do an inverse foruier transform by basically interpolating the polynomial to find the coefficient form. 



In [17]:
from scipy.interpolate import lagrange
x = [0,1,2]
res = lagrange(x,fs)
res = [float(x) for x in reversed(res.coefficients)]
assert(res == poly)

Okay so we have gone to foruier space and back to coefficient space. But why? 

Well turns out in evaluation space it is easier to do things like multiplicaion and division. So lets do that now. Lets take the polynomial $1 + 3 x^2$ and multiply it by itself. Lets do it both ways in fourier space and in coordinate space. 


In [18]:
res = [0]*len(poly)**2
expected_result = [1,0,6,0,9,0,0,0,0]
for i, coef1 in enumerate(poly):
    for j, coef2 in enumerate(poly):
        res[i+j] += coef1*coef2
assert(res == expected_result)


Okay so this took len(poly) ^ 2 operations. Lets try the fft version now. 

In [19]:
from plonk.poly import polynomial_eval

poly = [1,0,3]
fs = []
for i in range(0,9):
    fs.append(polynomial_eval(poly,i))


fs_res = [x*y for x,y in zip(fs,fs)]
x = range(0,9)
res = lagrange(x,fs_res)
res = [int(x) for x in reversed(res.coefficients)]
assert(res == expected_result)


Okay so this tool len(poly)*2 operations to do the multiplicaions. But there is still a problem. Because the transform and inverse both cost more than the saving in operations we need to find a faster way to do this fourier transform. That is where the fast in fast fourier transform comes in. 

Okay so firstly lets evaluate a polynomial over prime feild. You just have to take the result % p.

In [20]:
%pip install ethsnarks

You should consider upgrading via the '/Users/marcuspang/Desktop/Coding/pba/group-assignment-4/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
from plonk.fft.fft import polynomial_eval_prime
# this evaluations 3 + x**2 at position 0 % 5
assert(polynomial_eval_prime([3,0,1], 0, 5) == 3)
# this evaluations 3 + x**2 at position 1 % 5
assert(polynomial_eval_prime([3,0,1], 1, 5) == 4)
# this evaluations 3 + x**2 at position 2 % 5
assert(polynomial_eval_prime([3,0,1], 2, 5) == 2)

Every prime feild has something called roots of unity. Which is a number x such that $x^n == 1$. 

In [29]:
import sys
sys.setrecursionlimit(10000)

# python3 compatibility
try:
    foo = long
except:
    long = int

# The prime modulus of the field
field_modulus = 21888242871839275222246405745257275088696311157297823662689037894645226208583
# See, it's prime!
assert pow(2, field_modulus, field_modulus) == 2

# The modulus of the polynomial in this representation of FQ12
FQ12_modulus_coeffs = [82, 0, 0, 0, 0, 0, -18, 0, 0, 0, 0, 0] # Implied + [1]

# Extended euclidean algorithm to find modular inverses for
# integers
def inv(a, n):
    if a == 0:
        return 0
    lm, hm = 1, 0
    low, high = a % n, n
    while low > 1:
        r = high//low
        nm, new = hm-lm*r, high-low*r
        lm, low, hm, high = nm, new, lm, low
    return lm % n

# A class for field elements in FQ. Wrap a number in this class,
# and it becomes a field element.
class FQ():
    def __init__(self, n):
        if isinstance(n, self.__class__):
            self.n = n.n
        else:
            self.n = n % field_modulus
        assert isinstance(self.n, (int, long))

    def __add__(self, other):
        on = other.n if isinstance(other, FQ) else other
        return FQ((self.n + on) % field_modulus)

    def __mul__(self, other):
        on = other.n if isinstance(other, FQ) else other
        return FQ((self.n * on) % field_modulus)

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        on = other.n if isinstance(other, FQ) else other
        return FQ((on - self.n) % field_modulus)

    def __sub__(self, other):
        on = other.n if isinstance(other, FQ) else other
        return FQ((self.n - on) % field_modulus)

    def __div__(self, other):
        on = other.n if isinstance(other, FQ) else other
        assert isinstance(on, (int, long))
        return FQ(self.n * inv(on, field_modulus) % field_modulus)

    def __truediv__(self, other):
        return self.__div__(other)

    def __rdiv__(self, other):
        on = other.n if isinstance(other, FQ) else other
        assert isinstance(on, (int, long)), on
        return FQ(inv(self.n, field_modulus) * on % field_modulus)

    def __rtruediv__(self, other):
        return self.__rdiv__(other)

    def __pow__(self, other):
        if other == 0:
            return FQ(1)
        elif other == 1:
            return FQ(self.n)
        elif other % 2 == 0:
            return (self * self) ** (other // 2)
        else:
            return ((self * self) ** int(other // 2)) * self

    def __eq__(self, other):
        if isinstance(other, FQ):
            return self.n == other.n
        else:
            return self.n == other

    def __ne__(self, other):
        return not self == other

    def __neg__(self):
        return FQ(-self.n)

    def __repr__(self):
        return repr(self.n)

    @classmethod
    def one(cls):
        return cls(1)

    @classmethod
    def zero(cls):
        return cls(0)

# Utility methods for polynomial math
def deg(p):
    d = len(p) - 1
    while p[d] == 0 and d:
        d -= 1
    return d

def poly_rounded_div(a, b):
    dega = deg(a)
    degb = deg(b)
    temp = [x for x in a]
    o = [0 for x in a]
    for i in range(dega - degb, -1, -1):
        o[i] += temp[degb + i] / b[degb]
        for c in range(degb + 1):
            temp[c + i] -= o[c]
    return o[:deg(o)+1]

# A class for elements in polynomial extension fields
class FQP():
    def __init__(self, coeffs, modulus_coeffs):
        assert len(coeffs) == len(modulus_coeffs)
        self.coeffs = [FQ(c) for c in coeffs]
        # The coefficients of the modulus, without the leading [1]
        self.modulus_coeffs = modulus_coeffs
        # The degree of the extension field
        self.degree = len(self.modulus_coeffs)

    def __add__(self, other):
        assert isinstance(other, self.__class__)
        return self.__class__([x+y for x,y in zip(self.coeffs, other.coeffs)])

    def __sub__(self, other):
        assert isinstance(other, self.__class__)
        return self.__class__([x-y for x,y in zip(self.coeffs, other.coeffs)])

    def __mul__(self, other):
        if isinstance(other, (FQ, int, long)):
            return self.__class__([c * other for c in self.coeffs])
        else:
            assert isinstance(other, self.__class__)
            b = [FQ(0) for i in range(self.degree * 2 - 1)]
            for i in range(self.degree):
                for j in range(self.degree):
                    b[i + j] += self.coeffs[i] * other.coeffs[j]
            while len(b) > self.degree:
                exp, top = len(b) - self.degree - 1, b.pop()
                for i in range(self.degree):
                    b[exp + i] -= top * FQ(self.modulus_coeffs[i])
            return self.__class__(b)

    def __rmul__(self, other):
        return self * other

    def __div__(self, other):
        if isinstance(other, (FQ, int, long)):
            return self.__class__([c / other for c in self.coeffs])
        else:
            assert isinstance(other, self.__class__)
            return self * other.inv()

    def __truediv__(self, other):
        return self.__div__(other)

    def __pow__(self, other):
        if other == 0:
            return self.__class__([1] + [0] * (self.degree - 1))
        elif other == 1:
            return self.__class__(self.coeffs)
        elif other % 2 == 0:
            return (self * self) ** (other // 2)
        else:
            return ((self * self) ** int(other // 2)) * self

    # Extended euclidean algorithm used to find the modular inverse
    def inv(self):
        lm, hm = [1] + [0] * self.degree, [0] * (self.degree + 1)
        low, high = self.coeffs + [0], self.modulus_coeffs + [1]
        while deg(low):
            r = poly_rounded_div(high, low)
            r += [0] * (self.degree + 1 - len(r))
            nm = [x for x in hm]
            new = [x for x in high]
            assert len(lm) == len(hm) == len(low) == len(high) == len(nm) == len(new) == self.degree + 1
            for i in range(self.degree + 1):
                for j in range(self.degree + 1 - i):
                    nm[i+j] -= lm[i] * r[j]
                    new[i+j] -= low[i] * r[j]
            lm, low, hm, high = nm, new, lm, low
        return self.__class__(lm[:self.degree]) / low[0]

    def __repr__(self):
        return repr(self.coeffs)

    def __eq__(self, other):
        assert isinstance(other, self.__class__)
        for c1, c2 in zip(self.coeffs, other.coeffs):
            if c1 != c2:
                return False
        return True

    def __ne__(self, other):
        return not self == other

    def __neg__(self):
        return self.__class__([-c for c in self.coeffs])

    @classmethod
    def one(cls):
        return cls([1] + [0] * (cls.degree - 1))

    @classmethod
    def zero(cls):
        return cls([0] * cls.degree)

# The quadratic extension field
class FQ2(FQP):
    def __init__(self, coeffs):
        self.coeffs = [FQ(c) for c in coeffs]
        self.modulus_coeffs = [1, 0]
        self.degree = 2
        self.__class__.degree = 2

# The 12th-degree extension field
class FQ12(FQP):
    def __init__(self, coeffs):
        self.coeffs = [FQ(c) for c in coeffs]
        self.modulus_coeffs = FQ12_modulus_coeffs
        self.degree = 12
        self.__class__.degree = 12

In [30]:
# this evaluations 3 + x**2 at position 1
# from py_ecc.bn128.bn128_field_elements import FQ as GF
GF = FQ

from ethsnarks import field

def roots_of_unity(order):
     a = field.FQ(5)
     p = field.FQ(field.SNARK_SCALAR_FIELD)
     return [a**(i*(p-1)/order) for i in range(order)]

roots_2 = roots_of_unity(2)



So lets take these roots and square them see what happens. 

In [31]:
#assert(roots[0]**2 == roots[0])
assert(roots_2[1]**2 == 1 )
assert(roots_2[1]**3 == roots_2[1])
assert(roots_2[1]**4 == 1)


We see that they stay the same. Lets say that we have a polynomial above $3 + x  + x^2 + x^3 +  x^4$. If we want to evaluate it at a point $x^n = 1$ where $n = 2$. 

Lets say that we want to evaluate this polynomial at two points the naive thing to do is to  hen we can simply the above equation because we know that $x , x^3 == roots_2[1]$ and  

Now lets use the roots of unity to accelerate the evaluation of the polynomial above. 

So we can save doing the squaring above because x^2 == x^4 for these roots. That lets us do the evaluation by summing 3 + roots[0] + roots[0] and 3 + root[1] + root[1]. 



In [32]:
p = field.SNARK_SCALAR_FIELD
eval1 = 3 + roots_2[0] + roots_2[0] + roots_2[0] + roots_2[0]
eval2 = 3 + roots_2[1] + roots_2[0] + roots_2[1] + roots_2[0]

assert(polynomial_eval_prime([3,1,1,1,1], roots_2[0].n, p) == eval1)
assert(polynomial_eval_prime([3,1,1,1,1], roots_2[1].n, p) == eval2)
print(eval2)

3


So the last part is a bit strange. Since eval2 == 3 even tho roots[1] = a pretty big number. The reason for this is that roots[0] == -roots[1] read about negative numbers in finite fields https://vitalik.ca/general/2017/11/22/starks_part_2.html

So because a negative number squared is a positive number we only have to worry about half the domain for even powers. Check out the rest of https://vitalik.ca/general/2019/05/12/fft.html to fill in the details. 

Then write an fft that validates the testcase below

In [33]:
from plonk.fft.fft import fft
p = field.SNARK_SCALAR_FIELD
domain = roots_of_unity(8)
poly = [3,1,4,1,5,9,2,6]
result = []

p_x = fft(p, domain, poly)

for x in domain:
    result.append(polynomial_eval_prime(poly, x.n, p, 1, 0))

assert(p_x == result)


And the same for ifft

In [34]:
from plonk.fft.fft import ifft, fft

p = field.SNARK_SCALAR_FIELD
domain = roots_of_unity(8)
poly = [3,1,4,1,5,9,2,6]
result = []

p_x = fft(p, domain, poly)

result = ifft(p, domain, p_x)

assert(result == poly)


Okay thats fft done. We can do fast evaluations of polynomials. Do multiplicaion and stuff in fourier space and then convert back to coefficient space.

TODO: do this better

## Part x: Divison of polynomials 

Okay the next thing we need to do is find how to divide polynomials efficiently. We can work out a basic algorithm that does this but lets do it in fourier space since that will work out as being a bit faster. 

Take the polynomial $1 + x$ and square it using fft.

In [35]:
poly1 = [1,1,0,0,0,0,0,0]
poly2 = [1,1,0,0,0,0,0,0]


domain = roots_of_unity(8)

poly1_fs = fft(p,domain, poly1)
poly2_fs = fft(p,domain, poly2)

res = [x*y for x,y in zip(poly1_fs, poly2_fs)]

res = ifft(p,domain,res)

assert(res  == [1, 2, 1, 0, 0, 0, 0, 0])

Good now divide 
hint: remember to use field.FQ for it the division

In [36]:
poly1 = [1,2,1,0]
poly2 = [1,1,0,0]


domain = roots_of_unity(4)

poly1_fs = fft(p,domain, poly1)
poly2_fs = fft(p,domain, poly2)
res = []
for x,y in zip(poly1_fs, poly2_fs):
    x = field.FQ(x)
    y = field.FQ(y)
    res.append((x/y).n)
res = ifft(p,domain,res)
assert(res  == [1, 1, 0, 0])

Okay we can mutiply and divide in fft form which is cool. But what happens when we try and divide a polynomial by one that does not have a root? 

In [37]:
poly1 = [1,1,1,0]
poly2 = [1,1,0,0]


domain = roots_of_unity(8)

poly1_fs = fft(p,domain, poly1)
poly2_fs = fft(p,domain, poly2)
res = []
for x,y in zip(poly1_fs, poly2_fs):
    x = field.FQ(x)
    y = field.FQ(y)
    res.append((x/y).n)
res = ifft(p,domain,res)
print(res)


[13680151794899547015005983833359857890978056243232007815795651899964485005940, 13680151794899547015005983833359857890978056243232007815795651899964485005940, 13680151794899547012802023348211735969707399257288035113827103333255275613582, 13680151794899547012802023348211735969707399257288035113827103333255275613582]


TODO: find out the meaning of this polynomial



Okay so now we are able to quickly divide polynomials using fft what do we want this for?


Earlier we had polynomials that the verifier needed to multiply together in order to make sure they matched the data the prover sent. This work was square in the size of the polynomials which meant that our proof were not succinct. In order to make our proofs succinct we need to turn the verification into a bunch of polynomial evaluations. 



## Part x: is zero check

So we are working with polynomials and we want to be sure that a polynomial equals zero everywhere inside a domain that we care about. So what we do is make it so that the roots of an equation are equal to zero everywhere we care about. 
    
So we have a polynomial f(x) =  and we want to prove that it is zero at a bunch of places we care about say (x=1, x=2)
    
This is easy to do all we need to do is come up with a list of places that we care about (1,2,3) and rewrite our polynomial as the product of $z(x)*h(x) == f(x)$ where $z(x) = (x-1)(x-2)$

So we have the polynomial $f(x) = x^4-10x^3+35x^2-50x+24$ and we have the polynomial $(x-2)(x-3) = x^2 -5x + 6$

Use the polynomial division from above to calculate $h(x)$


    
    
    

In [38]:
fx = [24 , -50, 35, -10, 1,0,0,0]
zx = [6, -5, 1,0,0,0,0,0]

domain = roots_of_unity(8)
fx_fs = fft(p,domain, fx)
zx_fs = fft(p,domain, zx)
res = []
for x,y in zip(fx_fs, zx_fs):
    x = field.FQ(x)
    y = field.FQ(y)
    res.append((x/y).n)
res = ifft(p,domain,res)

#convert to negitive represtaionation

hx = []
for i in res:
    if i > p / 2:
        i = -(p - i)
    hx.append(i)

assert (hx == [4, -5, 1, 0,0,0,0,0])
# [4, -5, 1] == (x-1)(x-4)


todo: for some reason when i divide by x-1 it fails check this out.

Okay so now we have a way of checking that a poylnomial == 0 at every point. That we define. Next what we want to do is remove the requirement for the verifier to multiply two polynomials together and check that the results are equal. 

So we avoid this by instead of checking the polynomial at every point we check it at a single random point. So the verifier has polynomial 

$f(x)$ and $h(x)*z(x)$ so what we do is generate a random number (rand) and evaluate $f(x)$ and $h(x)$ , $z(x)$ at rant then we assert that $f(rand) == h(rand) * z(rand)$

In [39]:
rand = hash(str(fx))
assert(polynomial_eval_prime(fx,rand,p) == polynomial_eval_prime(zx, rand, p) * polynomial_eval_prime(hx, rand, p))

Turns out that this is the case for the majority of points on fx , zx and hx. Becuase we generate the rand point to evaluate fx we will only know that after it is created. So an attacker would have to spend a very long time to try and generate a random number that passes the test. So the the thinking is that this is enough to secure plonk and only check a single point. 

## Part x: Polynomial commitments

Okay so now we have succinct verification but these polynomials are really big. Here we will use polynomial commitments so that povers only need to commmit to their polynomials and open them at random points in order to convince a verifier. 

So we have a polynomial p = [5, 0, 2, 1] which is the same as $x^3 + 2x^2 + 0x + 5$ and we want to commit to it such that the commiter can open it at any point and convince a verifier that this is correct opening. 

So the first thing we are going to do is find a secret point that we will evaluate all our polynomials at. This secret point will be generated using a trusted setup. 

So to evaluate a polynomial we need to have a list of points x such that $(x , x^2, x^3, x^4 ... x^n)$

So we can easily evaluate these by taking a point in g_1 and multiplying it by $secret^n$ for n in range(1,n) where n is the length of the polynomial we want to commit to.

In [40]:
from py_ecc.bn128.bn128_curve import multiply, add, G1, G2 , FQ, neg
from py_ecc.bn128.bn128_pairing import pairing

from plonk.poly_commit import powers_of_tau
poly = [5,0,2,1]
secret = 1234
g1, g2 = powers_of_tau(secret, len(poly))

assert(g1 == [(1, 2), (5240721337203810155063577104887775964429040310352786870634285698927658009894, 8895618777946819312582035270689922760507554319433213576472857911545059134563), (6948375176493266981514273412529490307784532661099513628234608095184316458902, 18216600908385678342586247701838651916221324115124034844476606797464062286560), (13938208764793520230672803096399331546637544056917425701048277721869383690071, 17643137528866430994062863932942240127918427471357794446875411188797842000191), (11205499023060593757170025183462743313595252573361076357571209373922074319500, 10574276079570991118618272977314811771918167997289361521559698253334912778879)])


Okay so now we need to evaluate our polynomial at this point 

In [41]:
from plonk.poly_commit import poly_commit
p_commit = poly_commit(poly, g1)

assert(p_commit == (12751843803001521458487166996489945844735054316365547347615166726833607096115, 2774159934330533841376375186873158845895081394785984945802820943066298465301))

Okay so now we have evaluated our polynomial at a secret point. This we call the polynomial commitment. But how do we evaluate it at an arbitary point? 

So we use a subtract and divide trick to do this we say 

$\frac{p(secret) - a}{secret - z} = q(secret)$

Where a is the result when we evaluate p at z. Two problems present themselves 
1. Secret is secret so how can't we evaluate the denomiator 
2. We have to ensure that q(x) is a polynomial with no remainder.

We can easily solve 2 by using the commit and open trick from above. Where we commit to all polynomials and then AFTER that evaluate it at a single point. This means that if the points match with overwhelming probability the two polynomials are the same. 

We can solve 1 by doing the evaluation in the exponent of the pairing. So when we did our trusted setup we evaluated g1 which is the powers of all the x terms in the polynomial. So lets take a look at what we can do with pairings. First we can do homomorphic operations




In [42]:
from py_ecc.bn128.bn128_curve import multiply, add, G1, G2 , FQ, neg
from py_ecc.bn128.bn128_pairing import pairing


secret_1 = 1232
secret_2 = 123123
g_1 = multiply(G1,secret_1)
g_2 = multiply(G2,secret_2)

res = pairing(g_2, g_1)

g_1 = multiply(G1,secret_2)
g_2 = multiply(G2,secret_1)

# we flip it and it still equals its homomorphic.
assert(res == pairing(g_2, g_1))


Secondly we can encrypt variables such that you don't need to know them to compute with them. So what I can do is take g_2 and share that with someone. Everyone can then calculate g_1 time g_2 but they will never know what secret_2 is. That is what we need to do here. So during our trusted setup we calculate `g_2` which is a `g_2` generator times a secret and once we throw away the secret there is not way to find the secret again. 

Okay lets evaluate the polynomial `p` at a randome x using the polynomial commitment to check it. 

In [43]:


g1, g2 = powers_of_tau(1234, len(poly))

poly = [5,0,2,1]
p_commit = poly_commit(poly, g1)



We want to evaluate this where z = 6 so lets evaluate the result of the polynomial at that point. 

In [44]:
from plonk.fft.fft import polynomial_eval_prime

z = [6]
z_commit = poly_commit(z,g2)
print(p)
a = polynomial_eval_prime(poly, 6, p)
a = [a]
a_commit = poly_commit(a, g1)

# q = p /  [-a,1]
# todo use fft division from above to do this

q = [48,8,1]
q_commit = poly_commit(q, g1)

# this is the polynomial 0 + x
x = [0, 1]
x_commit = poly_commit(x, g2)


# evaluate the polynomials
# p - a = q*(x-z)
p = field.SNARK_SCALAR_FIELD

lhs = polynomial_eval_prime(poly,z[0],p) - polynomial_eval_prime(a,z[0],p)
rhs =  polynomial_eval_prime(q,z[0],p) * (polynomial_eval_prime(x,z[0],p) - z[0])

assert(lhs == rhs)

21888242871839275222246405745257275088548364400416034343698204186575808495617


Okay when we evaluate the polynomial it works but lets use the pairing check.

In [45]:
lhs = pairing(G2, add(p_commit,neg(a_commit)))
rhs = pairing(add(x_commit,neg(z_commit)), q_commit)

assert(lhs == rhs)

TODO: Go a little slower here for the last step. Possibly break it out into calculating a, q, x , z and then do the pairing check. 

## Part x: Putting it all together
Okay lets list the prover work , the verifier work and the setup work.

### SETUP: 

#### Gate constraint

#### Copy constraint

### Prover:

#### Gate constaints
a, b ,c polynomial

#### copy constraints

#### polynomial commitment openings

### Verifier:

TODO: change zx to be the roots of unity version and not the naive (x-1)(x-2) this will make the zx polynoiail small and easy to verifiy. 

## Part x: Bounties


Here are a bunch fo follow on ideas, the bounty is to add another section to this tutorial explaining the following

1. Fix some TODOs
2. Implement plookup
3. Implement range proofs for plonk
4. Implement build programming language on top of this
5. Explore custom constraints 
6. There are a few security bugs in what we have implemented so far. Can you find them / fix them?
7. Build semaphore on top of this

TODO: Find out about adding bounties or rewards to these tasks. 